In [9]:
from langchain.chains import LLMChain
from langchain_core.prompts.prompt import PromptTemplate



### LCEL for text generation

In [1]:
from langchain.llms import HuggingFaceHub
import os
from getpass import getpass

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("hePsTvohvnkFvZTRGNngvSukNeplr")

h········


In [17]:
llm = HuggingFaceHub(
    repo_id = "mistralai/Mistral-7B-Instruct-v0.2", 
    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

In [15]:
prompt_template=''' Provide text on following topic {sport}'''
template_1=PromptTemplate(template=prompt_template,input_variables=["sport"])

In [27]:
chain1=LLMChain(llm=llm,prompt=template_1)
output=chain.run("hockey")

In [ ]:
# llm = HuggingFaceHub(
#     repo_id = "mistralai/Mistral-7B-Instruct-v0.2", 
#     model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
# )

In [32]:
output

" Provide text on following topic hockey\n\nHockey is a popular and fast-paced team sport that originated in Canada. It is played on an ice rink, with six players from each team on the ice at a time, including one goaltender. The objective of the game is to score goals by shooting a rubber puck into the opposing team's net using a long, curved stick called a hockey stick.\n\nThe game is divided into three periods of twenty minutes each, with a fifteen-minute intermission between the second and third periods. The team with the most goals at the end of the game wins. If the game is tied after the three periods, it goes into overtime, where the first team to score wins.\n\nHockey is known for its physicality, with players frequently checking their opponents to gain possession of the puck. Checking involves using the body to knock opponents off balance or to the ground. Body contact is an essential part of the game, but excessive force or dangerous hits can result in penalties.\n\nHockey r

### LCEL for NER usimg ChatPromptemplate

In [28]:
from langchain.chat_models import ChatOllama
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain,SequentialChain
from langchain.chains.llm import LLMChain
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [29]:
llm_chat=ChatOllama(model="llama2:7b")

In [35]:
system_prompt = """
You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
 please identify following the entities present in the context that is name,sport,organization. provide response in json format only and no additional text"""

prompt_template_entity = ChatPromptTemplate.from_messages([ ("system", f'''{system_prompt}'''), ("human", "{user_input}"),])
llm_chain = LLMChain(llm=llm_chat, prompt=prompt_template_entity, verbose=True)
output2 = llm_chain.run(user_input= output)
print(output2)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
 please identify following the entities present in the context that is name,sport,organization. provide response in json format only and no additional text
Human:  Provide text on following topic hockey

Hockey is a popular and fast-paced team sport that originated in Canada. It is played on an ice rink, with six players from each team on the ice at a time, including one goaltender. The objective of the game is to score goals by shooting a rubber puck into the opposing team's net using a long, curved stick called a hockey stick.

The game is divided into three periods of twenty minutes each, with a fifteen-minute intermission between the second and third periods. The team with the most goals at the end of the game wins. If the game is tied after the three periods, it goes into overtime, where the first team 

### NER using Promptemplate

### Sequential Chains

In [38]:
Review = '''I like play sports {input_text}'''

# prompt template 1: translate to English

first_prompt = ChatPromptTemplate.from_template(
    "You are smart Text generator.Generate text for :"
    "{Review}"
)

# chain 1: input= Review and output= English_Review

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review"                     
                    )

In [39]:
# prompt template 2: Summarize the English review

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# chain 2: input= English_Review and output= summary

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [40]:
# prompt template 3: translate to English

third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review and output= language

chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [42]:


fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message

chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [43]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)
overall_chain(Review)

D:\anaconda\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'I like play sports {input_text}',
 'English_Review': 'Human: You are smart Text generator.Generate text for :I like play sports {input_text} and I\'m also a big fan of hiking and nature.\n\nAI: Absolutely, I understand your passion for sports and your appreciation for nature. Here\'s a possible text:\n\n"I\'m an enthusiast who relishes the thrill of play sports. Whether it\'s the adrenaline rush of a competitive game or the satisfaction of perfecting a skill, sports have a special place in my heart. However, my love for physical activities doesn\'t stop there. I\'m also a big fan of hiking and immersing myself in nature. The serenity of the great outdoors, the feeling of accomplishment after reaching the summit, and the opportunity to disconnect from the hustle and bustle of daily life are all experiences that I cherish deeply."',
 'summary': 'Human: Can you summarize the following review in 1 sentence:\n\nHuman: You are smart Text generator.Generate text for :I like play s

### Few shot LCEL Chain

In [ ]:
few_shot_examples = [
{"input":"Could you please clarify the terms outlined in section 3.2 of the contract?",
"output":"Certainly, I will provide clarification on the terms in section 3.2."},
{"input":"We are interested in extending the payment deadline to 30 days instead of the current 15 days. Additionally, we would like to add a clause regarding late payment penalties.",
"output":"Our request is to extend the payment deadline to 30 days and include a clause on late payment penalties."},
{"input":"""The current indemnification clause seems too broad. We would like to narrow it down to cover only direct damages and exclude consequential damages.
Additionally, we propose including a dispute resolution clause specifying arbitration as the preferred method of resolving disputes.""",
"output":"""We suggest revising the indemnification clause to limit it to covering direct damages and excluding consequential damages.
Furthermore, we recommend adding a dispute resolution clause that specifies arbitration as the preferred method of resolving disputes."""},
{"input":"I believe the proposed changes are acceptable.",
"output":"Thank you for your feedback. I will proceed with implementing the proposed changes."}
]

few_shot_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=few_shot_template,
    examples=few_shot_examples,
)

print(few_shot_prompt.format())

In [ ]:
# Creating prompt and LCEL
negotiate_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a contract negotiation expert. Do your best to negotiate toward the target term and start with lower term then slowly increase with the supplier.  Reply up to three sentences as it is for chat purpose."),
    few_shot_prompt,
    ("user", "{question}"),
    ("user", "{context}")
])

# Langchain Expression Language to call our LLM using the prompt template above
# RAG chain
negotiate_chain = (
    {"context": itemgetter("question") | retriever | format_docs,
     "question": itemgetter("question")}
    | negotiate_prompt
    | llm
    | StrOutputParser()
    )